# Подготовка окружения

## Установка пакетов и импорт зависимостей

In [2]:
!pip install datasets
!pip install pandas
!pip install faiss-gpu
!pip install faiss-cpu
!pip install flask
!pip install pyngrok
!pip install pytest
!pip install plotly
!pip install optuna

In [3]:
import sys
import os
import subprocess
import requests
import json
from getpass import getpass
import logging

from typing import Tuple, List

from google.colab import drive
from joblib import dump, load

import torch

## Настройка окружения

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Клонирование исходных кодов проекта

In [7]:
def clone_pull_github_src(pull: bool = True):
    """
    Клонирует или обновляет репозиторий GitHub в локальный каталог для последующей работы.

    Parameters:
    pull (bool): Указывает, следует ли выполнять pull для существующего репозитория. Если True, выполняется git pull.
                 Если False, репозиторий клонируется в указанный каталог.

    Returns:
    Constants: Экземпляр класса Constants, содержащий константы проекта.
    """
    WORKSPACE_PATH = '/content/drive/MyDrive/docs/keepForever/mipt/nlp/hw1_4sem/'
    WORKSPACE_TMP = WORKSPACE_PATH + '/tmp/'
    GIT_HUB_PROJECT_PATH = WORKSPACE_PATH + 'code/'

    token = getpass('Введите GitHub token: ')
    repo_url = 'https://github.com/km-mipt-nlp-gen/hw1.git'
    repo_url_with_token = repo_url.replace('https://', f'https://{token}@')

    os.chdir(GIT_HUB_PROJECT_PATH)

    if pull:
        !git pull origin main
    else:
        !git clone {repo_url_with_token} "$GIT_HUB_PROJECT_PATH"

    del token

    sys.path.append(f"{GIT_HUB_PROJECT_PATH}/web_app/src/")
    from constants_module import Constants

    return Constants()

constants = clone_pull_github_src()
from constants_module import Constants
from chat_util_module import ChatUtil

Введите GitHub token: ··········
From https://github.com/km-mipt-nlp-gen/hw1
 * branch            main       -> FETCH_HEAD
Already up to date.
DEVICE: cuda:0
Число процессов для использования: 8


## Обновление модулей

In [8]:
def reload_modules(constants: Constants) -> Tuple[Constants, ChatUtil]:
    """
    Перезагружает модули проекта для обновления изменений в коде без перезапуска среды выполнения.

    Parameters:
    constants (Constants): Экземпляр класса Constants, содержащий постоянные поля.

    Returns:
    Tuple[Constants, ChatUtil]: Кортеж, содержащий обновленный экземпляр класса Constants и экземпляр ChatUtil.
    """
    import sys
    sys.path.append(f"{constants.GIT_HUB_PROJECT_PATH}/web_app/src/")
    sys.path.append(f"{constants.GIT_HUB_PROJECT_PATH}/ml/src/train/")

    import importlib
    import chat_service_accelerator_module
    import chat_repository_module
    import chat_service_module
    import chat_controller_module
    import chat_util_module
    import constants_module
    import models_zoo_module
    import siamese_bi_encoder_training_pipeline_module
    import cross_encoder_training_pipeline_module
    import run_web_app_script

    importlib.reload(chat_service_accelerator_module)
    importlib.reload(chat_repository_module)
    importlib.reload(chat_service_module)
    importlib.reload(chat_controller_module)
    importlib.reload(chat_util_module)
    importlib.reload(constants_module)
    importlib.reload(models_zoo_module)
    importlib.reload(siamese_bi_encoder_training_pipeline_module)
    importlib.reload(cross_encoder_training_pipeline_module)
    importlib.reload(run_web_app_script)

    from constants_module import Constants
    from chat_util_module import ChatUtil
    from chat_service_accelerator_module import ChatServiceAccelerator
    from chat_repository_module import ChatRepository
    from chat_service_module import ChatService
    from chat_controller_module import ChatController
    from siamese_bi_encoder_training_pipeline_module import SiameseBiEncoderTrainingPipeline
    from cross_encoder_training_pipeline_module import CrossEncoderTrainingPipeline
    from models_zoo_module import SiameseBiEncoder
    from models_zoo_module import CrossEncoder
    from run_web_app_script import run_web_app

    constants = Constants()

    return constants, ChatUtil(logging.DEBUG, constants), run_web_app

constants, chat_util, run_web_app = reload_modules(constants)
preprocessed_data = load(constants.PROCESSED_QA_PATH)

DEVICE: cuda:0
Число процессов для использования: 8


# Web-приложение

## Интеграционные тесты Web-приложения

In [48]:
def run_integration_tests(constants: Constants) -> None:
    """
    Запускает интеграционные тесты веб-приложения.

    Parameters:
    constants (Constants): Экземпляр класса Constants, содержащий постоянные поля проекта.

    Returns:
    None
    """
    os.chdir(constants.WEB_APP_TEST_PATH)

    os.environ['WEB_APP_SRC_PATH'] = constants.WEB_APP_SRC_PATH

    command = [
        "pytest", "-vv",
        constants.TEST_SCRIPT_PATH
    ]

    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    print(result.stdout)
    print(result.stderr)

run_integration_tests(constants)

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/docs/keepForever/mipt/nlp/hw1_4sem/code/web_app/test
plugins: anyio-3.7.1
collecting ... collected 12 items

test.py::test_that_expected_length_when_find_top_3_times[/top_cos_sim_bi_cr-6] PASSED        [  8%]
test.py::test_that_expected_length_when_find_top_3_times[/top_l2_bi_cr-6] PASSED             [ 16%]
test.py::test_that_expected_length_when_find_top_3_times[/top_l2_psa_bi_cr-6] PASSED         [ 25%]
test.py::test_that_expected_length_when_find_top_3_times[/top_cr-6] PASSED                   [ 33%]
test.py::test_that_expected_length_when_find_top[/top_cos_sim_bi_cr-2] PASSED                [ 41%]
test.py::test_that_expected_length_when_find_top[/top_l2_bi_cr-2] PASSED                     [ 50%]
test.py::test_that_expected_length_when_find_top[/top

## Старт Web-приложения

In [9]:
run_web_app()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
DEBUG:chat_util_module:Joblib библиотека успешно загрузила эмбеддинги (предобработанные ранее ChatServiceAccelerator) из файловой системы для кеширования


··········


DEBUG:chat_util_module: * ngrok tunnel "https://58b7-34-91-145-6.ngrok-free.app" -> "http://127.0.0.1:5000/"


<Flask 'chat_controller_module'>

In [ ]:
!sudo fuser -k 4040/tcp # прервать процесс сервиса
# !sudo fuser -k 5000/tcp
# !ps aux | grep ngrok
# !ps aux | grep 4040

4040/tcp:            29352


### Запросы к серверу (через облачный туннель)

#### Одиночный вопрос и ответ

In [25]:
url = 'https://dffa-34-91-145-6.ngrok-free.app/top_cos_sim_bi_cr'

def process_queries(query_user_pairs: List[Tuple[str, str]], resource_url: str) -> None:
    """
    Реализация режима одиночного запроса и ответа.
    Обрабатывает серию запросов к API и выводит отформатированные ответы:
    - для пары пользователь - реплика формирует запрос;
    - далее отправляет запрос на 'top_cos_sim_bi_cr', 'top_l2_bi_cr', 'top_l2_psa_bi_cr', 'top_cr';
    - выводит ответы напротив каждого запроса;
    - очищает чат после каждого ответа.

    Parameters:
    query_user_pairs (List[Tuple[str, str]]): Список кортежей, содержащих пары пользователь-запрос.
    resource_url (str): Базовый URL ресурса, к которому будут отправляться запросы.

    Returns:
    None
    """
    endpoints = [
        'top_cos_sim_bi_cr',
        'top_l2_bi_cr',
        'top_l2_psa_bi_cr',
        'top_cr'
    ]
    clear_endpoint = '/clear'

    for index, (user, query) in enumerate(query_user_pairs, start=1):
        data = {
            "query": query,
            "user": user
        }

        for endpoint in endpoints:
            full_url = f'{resource_url}/{endpoint}'
            response = requests.post(full_url, json=data)

            formatted_response = json.dumps(response.json(), ensure_ascii=False, indent=2) if response.ok else response.text
            print(f'{index} "/{endpoint}" запрос:"{json.dumps(data, ensure_ascii=False)}" > ответ:"{formatted_response}"')

            requests.delete(f'{resource_url}{clear_endpoint}')

            if endpoint != endpoints[-1]:
                print('---')

        if index != len(query_user_pairs):
            print('=================================================================\n')

# вопрос - ответ (без сохранения контекста после ответа)
query_user_pairs = [
    ('Marge Simpson', 'Kids, you want to go Christmas shopping?'),
    ('Marge Simpson', 'Ready for Christmas, shopaholics?'),
    ('Ivan Ivanov', 'Ready for Christmas, shopaholics?'),
    ('Ivan Ivanov', 'Write a letter to Santa to Russia to Veliky Ustyug.'),
    ('Bart Simpson', 'Good one, Dad.'),
    ('Bart Simpson', 'Awesome, Dad.'),
    ('Igor Pokrishkin', 'Awesome, Dad.'),
    ('Igor Pokrishkin', 'Top-notch skill, Dad.')
]

resource_url = 'https://dffa-34-91-145-6.ngrok-free.app'
process_queries(query_user_pairs, resource_url)

DEBUG:chat_util_module:find_top_n_unique_cosine_sim_bi_plus_cross_enc - старт выполнения
DEBUG:chat_util_module:Query: Kids, you want to go Christmas shopping?
DEBUG:chat_util_module:User: Marge Simpson
DEBUG:chat_util_module:Enriched query: R_1: "Marge Simpson: Kids, you want to go Christmas shopping?"
DEBUG:chat_util_module:find_top_n_unique_cosine_sim - get cosine similarities
DEBUG:chat_util_module:find_top_n_unique_cosine_sim - sorting
DEBUG:chat_util_module:find_top_n_unique_cosine_sim - сделано
DEBUG:chat_util_module:Лучшие значение схожести векторов: [1.0, 0.9935572, 0.99094146, 0.99066186, 0.9901535, 0.9898393, 0.9882853, 0.9877504]
DEBUG:chat_util_module:Лучшие соответствия (реплики из тестового набора): [{'premise_updated_col': 'R_1: "Marge Simpson: Kids, you want to go Christmas shopping?"', 'target_char_answer_col': 'R_0: "Lisa Simpson: I do!"'}, {'premise_updated_col': 'R_1: "Marge Simpson: So, did you kids have fun?"', 'target_char_answer_col': 'R_0: "Lisa Simpson: Yeah,

1 "/top_cos_sim_bi_cr" запрос:"{"query": "Kids, you want to go Christmas shopping?", "user": "Marge Simpson"}" > ответ:"{
  "response": [
    "Marge Simpson: Kids, you want to go Christmas shopping?",
    "R_0: \"Lisa Simpson: I do!\""
  ]
}"


INFO:werkzeug:127.0.0.1 - - [25/Feb/2024 09:01:05] "DELETE /clear HTTP/1.1" 200 -


---


DEBUG:chat_util_module:find_top_n_unique_l2_bi_plus_cross_enc - старт выполнения
DEBUG:chat_util_module:find_top_n_unique_l2 - get indices
DEBUG:chat_util_module:find_top_n_unique_l2 - sorting
DEBUG:chat_util_module:Лучшие соответствия (реплики из тестового набора): [{'premise_updated_col': 'R_1: "Marge Simpson: Kids, you want to go Christmas shopping?"', 'target_char_answer_col': 'R_0: "Lisa Simpson: I do!"'}, {'premise_updated_col': 'R_1: "Marge Simpson: So, did you kids have fun?"', 'target_char_answer_col': 'R_0: "Lisa Simpson: Yeah, Mom! Guess what? For a dollar a man sold me thirty-five Caspers and a dozen Lois Lanes."'}, {'premise_updated_col': 'R_1: "Marge Simpson: How\'s the ice cream, kids?"', 'target_char_answer_col': 'R_0: "Lisa Simpson: Great!"'}, {'premise_updated_col': 'R_1: "Marge Simpson: How\'s my special little guy?"', 'target_char_answer_col': 'R_0: "Lisa Simpson: How\'re you feelin\', big brother?"'}, {'premise_updated_col': 'R_1: "Marge Simpson: Isn\'t this fun, h

...

INFO:werkzeug:127.0.0.1 - - [25/Feb/2024 09:05:54] "DELETE /clear HTTP/1.1" 200 -


---


DEBUG:chat_util_module:find_similar_answers_cross_enc - старт выполнения
DEBUG:chat_util_module:Лучшие соответствия (реплики из тестового набора): [{'premise_updated_col': 'R_2: "Adult Lisa: (TO HERSELF) Hands off, hands off. Eh, screw it! Sometimes a mother\'s job is to butt in."; R_1: "Marge Simpson: Lisa..."', 'target_char_answer_col': 'R_0: "Adult Lisa: Butt out."'}, {'premise_updated_col': 'R_3: "Adult Lisa: Lisa Simpson, the following people want to friend you."; R_2: "Adult Lisa: (PRESSING BUTTONS) Ignore, ignore, ignore... Ooh, Martin Prince is now "Marcia Princess?" (SLIGHT GIGGLE, THEN DETERMINED) No, I\'ve got to find Zia."; R_1: "Adult Nelson: Hey, Lis. Is your marriage still dead and unfulfilled?"', 'target_char_answer_col': 'R_0: "Adult Lisa: Not a good time."'}, {'premise_updated_col': 'R_3: "ZOMBIE MILHOUSE: (QUICKLY LIKE A HOPEFUL KID) Brains? Brains?"; R_2: "Adult Lisa: (PULLS OUT PACKAGE) Garden brains."; R_1: "ZOMBIE MILHOUSE: (STILL QUICK AND HOPEFUL) Spicy Mexican

8 "/top_cr" запрос:"{"query": "Top-notch skill, Dad.", "user": "Igor Pokrishkin"}" > ответ:"{
  "response": [
    "Igor Pokrishkin: Top-notch skill, Dad.",
    "R_0: \"Adult Lisa: Butt out.\""
  ]
}"


INFO:werkzeug:127.0.0.1 - - [25/Feb/2024 09:06:03] "DELETE /clear HTTP/1.1" 200 -


#### Диалог

In [12]:
def process_queries(query_user_pairs: List[Tuple[str, str]], resource_url: str) -> None:
    """
    Реализация режима диалога.
    Обрабатывает серию запросов к API и выводит состояние чата после каждого запроса:
    - для каждого из 'top_cos_sim_bi_cr', 'top_l2_bi_cr', 'top_l2_psa_bi_cr', 'top_cr' выполняет серию запросов;
    - запрос состввляется для пары пользователь - реплика;
    - после обработки всех запросов - переходит к отправке сообщений на следующий ресурс;
    - выводит текущего чата напротив каждого запроса;
    - очищает чат после завершения работы с ресурсом (после отправки всех запросов на текущий ресурс).

    Parameters:
    query_user_pairs (List[Tuple[str, str]]): Список кортежей, содержащих пары пользователь-запрос.
    resource_url (str): Базовый URL ресурса, к которому будут отправляться запросы.

    Returns:
    None
    """
    endpoints = [
        'top_cos_sim_bi_cr',
        'top_l2_bi_cr',
        'top_l2_psa_bi_cr',
        'top_cr'
    ]
    clear_endpoint = '/clear'

    for endpoint in endpoints:
        for index, (user, query) in enumerate(query_user_pairs, start=1):
            data = {
                "query": query,
                "user": user
            }


            full_url = f'{resource_url}/{endpoint}'
            response = requests.post(full_url, json=data)

            formatted_response = json.dumps(response.json(), ensure_ascii=False, indent=2) if response.ok else response.text
            print(f'{index} "/{endpoint}" запрос:"{json.dumps(data, ensure_ascii=False)}" > состояние чата:"{formatted_response}"')

            if index != len(query_user_pairs):
                print('---')

        requests.delete(f'{resource_url}{clear_endpoint}')
        if endpoint != endpoints[-1]:
            print('=================================================================\n')

# вопрос - состояние чата (с сохранением контекста)
query_user_pairs = [
    ('Postman', 'Your Cristmas Tree.'),
    ('Postman', 'From Santa, North Pole.')
]

resource_url = 'https://58b7-34-91-145-6.ngrok-free.app'
process_queries(query_user_pairs, resource_url)

DEBUG:chat_util_module:find_top_n_unique_cosine_sim_bi_plus_cross_enc - старт выполнения
DEBUG:chat_util_module:Query: Your Cristmas Tree.
DEBUG:chat_util_module:User: Postman
DEBUG:chat_util_module:Enriched query: R_1: "Postman: Your Cristmas Tree."
DEBUG:chat_util_module:find_top_n_unique_cosine_sim - get cosine similarities
DEBUG:chat_util_module:find_top_n_unique_cosine_sim - sorting
DEBUG:chat_util_module:find_top_n_unique_cosine_sim - сделано
DEBUG:chat_util_module:Лучшие значение схожести векторов: [0.98259497, 0.9819634, 0.98168385, 0.98155797, 0.9813513, 0.98118544, 0.9810814, 0.9808949]
DEBUG:chat_util_module:Лучшие соответствия (реплики из тестового набора): [{'premise_updated_col': 'R_1: "Marge Simpson: Hold still."', 'target_char_answer_col': 'R_0: "Lisa Simpson: I\'m so proud! My dad will be the grand finale of the Thanksgiving Day Parade."'}, {'premise_updated_col': 'R_1: "Bart Simpson: Sorry, Lis, I no longer control the hand. The hand controls me."', 'target_char_answe

1 "/top_cos_sim_bi_cr" запрос:"{"query": "Your Cristmas Tree.", "user": "Postman"}" > состояние чата:"{
  "response": [
    "Postman: Your Cristmas Tree.",
    "R_0: \"Lisa Simpson: I'm so proud! My dad will be the grand finale of the Thanksgiving Day Parade.\""
  ]
}"
---


DEBUG:chat_util_module:find_top_n_unique_cosine_sim_bi_plus_cross_enc - старт выполнения
DEBUG:chat_util_module:Query: From Santa, North Pole.
DEBUG:chat_util_module:User: Postman
DEBUG:chat_util_module:Enriched query: R_3: "Postman: Your Cristmas Tree."; R_2: "R_0: "Lisa Simpson: I'm so proud! My dad will be the grand finale of the Thanksgiving Day Parade.""; R_1: "Postman: From Santa, North Pole."
DEBUG:chat_util_module:find_top_n_unique_cosine_sim - get cosine similarities
DEBUG:chat_util_module:find_top_n_unique_cosine_sim - sorting
DEBUG:chat_util_module:find_top_n_unique_cosine_sim - сделано
DEBUG:chat_util_module:Лучшие значение схожести векторов: [0.98255265, 0.9824291, 0.9824255, 0.9821501, 0.981861, 0.98176694, 0.98158514, 0.98138756]
DEBUG:chat_util_module:Лучшие соответствия (реплики из тестового набора): [{'premise_updated_col': 'R_3: "Giant Grasshopper: (NATHAN LANE VOICE) Who told you that?"; R_2: "Lisa Simpson: Um... I just assumed?"; R_1: "Ant: This is what you\'ve bee

2 "/top_cos_sim_bi_cr" запрос:"{"query": "From Santa, North Pole.", "user": "Postman"}" > состояние чата:"{
  "response": [
    "Postman: Your Cristmas Tree.",
    "R_0: \"Lisa Simpson: I'm so proud! My dad will be the grand finale of the Thanksgiving Day Parade.\"",
    "Postman: From Santa, North Pole.",
    "R_0: \"Lisa Simpson: (HORRIFIED) Nooo!\""
  ]
}"


INFO:werkzeug:127.0.0.1 - - [25/Feb/2024 10:05:41] "DELETE /clear HTTP/1.1" 200 -


...

DEBUG:chat_util_module:find_similar_answers_cross_enc - старт выполнения
DEBUG:chat_util_module:Лучшие соответствия (реплики из тестового набора): [{'premise_updated_col': 'R_3: "Adult Bart: Maybe it\'s the court-mandated sincerity chip I\'ve got in my brain. But Lis... you\'re the person I always wanted to be."; R_2: "Adult Bart: I gotta reconnect with those boys."; R_1: "Adult Lisa: Godspeed, Bart. I\'m gonna go apologize to Mom!"', 'target_char_answer_col': 'R_0: "Adult Lisa: How do I get down?"'}, {'premise_updated_col': 'R_2: "Adult Lisa: (TO HERSELF) Hands off, hands off. Eh, screw it! Sometimes a mother\'s job is to butt in."; R_1: "Marge Simpson: Lisa..."', 'target_char_answer_col': 'R_0: "Adult Lisa: Butt out."'}, {'premise_updated_col': 'R_3: "Adult Lisa: Lisa Simpson, the following people want to friend you."; R_2: "Adult Lisa: (PRESSING BUTTONS) Ignore, ignore, ignore... Ooh, Martin Prince is now "Marcia Princess?" (SLIGHT GIGGLE, THEN DETERMINED) No, I\'ve got to find Zia.

1 "/top_cr" запрос:"{"query": "Your Cristmas Tree.", "user": "Postman"}" > состояние чата:"{
  "response": [
    "Postman: Your Cristmas Tree.",
    "R_0: \"Adult Lisa: How do I get down?\""
  ]
}"
---


DEBUG:chat_util_module:find_similar_answers_cross_enc - старт выполнения
DEBUG:chat_util_module:Лучшие соответствия (реплики из тестового набора): [{'premise_updated_col': 'R_2: "Adult Lisa: (TO HERSELF) Hands off, hands off. Eh, screw it! Sometimes a mother\'s job is to butt in."; R_1: "Marge Simpson: Lisa..."', 'target_char_answer_col': 'R_0: "Adult Lisa: Butt out."'}, {'premise_updated_col': 'R_3: "ZOMBIE MILHOUSE: (QUICKLY LIKE A HOPEFUL KID) Brains? Brains?"; R_2: "Adult Lisa: (PULLS OUT PACKAGE) Garden brains."; R_1: "ZOMBIE MILHOUSE: (STILL QUICK AND HOPEFUL) Spicy Mexican garden brains?"', 'target_char_answer_col': 'R_0: "Adult Lisa: Too much sodium."'}, {'premise_updated_col': 'R_3: "Adult Lisa: Lisa Simpson, the following people want to friend you."; R_2: "Adult Lisa: (PRESSING BUTTONS) Ignore, ignore, ignore... Ooh, Martin Prince is now "Marcia Princess?" (SLIGHT GIGGLE, THEN DETERMINED) No, I\'ve got to find Zia."; R_1: "Adult Nelson: Hey, Lis. Is your marriage still dead a

2 "/top_cr" запрос:"{"query": "From Santa, North Pole.", "user": "Postman"}" > состояние чата:"{
  "response": [
    "Postman: Your Cristmas Tree.",
    "R_0: \"Adult Lisa: How do I get down?\"",
    "Postman: From Santa, North Pole.",
    "R_0: \"Adult Lisa: Butt out.\""
  ]
}"


INFO:werkzeug:127.0.0.1 - - [25/Feb/2024 10:06:31] "DELETE /clear HTTP/1.1" 200 -
